In [7]:
import numpy as np
from scipy.stats import norm

In [39]:
def mlmc(N0, eps, mlmc_l, alpha_0, beta_0, gamma):
    alpha = np.max(0, alpha_0)
    beta = np.max(0, beta_0)
    L = 2
    Nl = np.zeros(3)
    suml = np.zeros((2,3))
    dNl = np.ones(3) * N0
    while np.sum(dNl) > 0 :
        for l in range(0,L):
            if(dNl[l] > 0):
                sums = mlmc_l(l, int(dNl[l]))
                Nl[l] = Nl[l] + dNl[l]
                suml[0, l] = suml[0, l] + sums[0]
                suml[1, l] = suml[1, l] + sums[1]
        ml = np.abs(suml[0,:]/Nl)
        Vl = np.maximum(0, suml[1, :]/Nl - ml**2)
        for l in range(2, L):
            ml[l] = np.maximum(ml[l], 0.5*ml[l-1]/2^alpha)
            Vl[l] = np.maximum(Vl[l], 0.5*Vl[l-1]/2^beta)
        if alpha_0 <= 0:
            # Compute alpha using a log-log linear fit
            idx = np.arange(1, L + 1)
            slope, intercept = np.polyfit(np.log(idx), np.log(ml[1:]), 1)
            alpha = max(0.5, -slope)

        if beta_0 <= 0:
            # Compute beta using a log-log linear fit
            idx = np.arange(1, L + 1)
            slope, intercept = np.polyfit(np.log(idx), np.log(Vl[1:]), 1)
            beta = max(0.5, -slope)
            
        Cl = 2**(gamma * np.arange(L + 1))
        sum_term = np.sum(np.sqrt(Vl * Cl))
        Ns = np.ceil(2 * np.sqrt(Vl / Cl) * sum_term / eps**2)
        dNl = np.maximum(0, Ns - Nl)
        
        range_values = np.arange(-2, 1)
        if np.sum(dNl > 0.01 * Nl) == 0:
            rem = np.max(ml[L + 1 + range_values] * 2**(alpha * range_values)) / (2**alpha - 1)
            if rem > eps / np.sqrt(2):
                L += 1
                Vl[L + 1] = Vl[L] / 2**beta
                Nl[L + 1] = 0
                suml[0:4, L + 1] = 0
                Cl = 2**(gamma * np.arange(L + 1))
                sum_term = np.sum(np.sqrt(Vl / Cl) * np.sqrt(Vl * Cl))
                Ns = np.ceil(2 * sum_term / eps**2)
                dNl = np.maximum(0, Ns - Nl)
    P = np.sum(suml[0, :] / Nl)
    return P

# Testing Zone

### Geometric Brownian motion
Let S be an asset with the following dynamic \\
$$
\frac{dS_t}{S_t} = r dt + \sigma dWt
$$ 
with 
$$
r = 0.05\\
\sigma = 0.2\\
S_0 = 100
$$
Let's consider a european call option on S
$$
Payoff = (S_T - K)_+\\
K = 100\\
T = 1\\
$$


In [40]:
def eu_call_BS(S, T, K, r, q, sigma,t=0):
    d1 = (np.log(S/K) + (r - q + sigma**2/2)*(T-t)) / (sigma*np.sqrt(T-t))
    d2 = d1 - sigma* np.sqrt(T-t)
    return S*np.exp(-q*(T-t)) * norm.cdf(d1) - K * np.exp(-r*(T-t))* norm.cdf(d2)

In [41]:
S = 100
T = 1
K = 100
r = 0.05
q = 0
sigma = 0.2
th_price = eu_call_BS(S, T, K, r, q, sigma)
print("Theoritical Price = %s"%(th_price))

Theoritical Price = 10.450583572185565


Now lets try to price our option with Monte Carlo Using an Euler discretization for the SDE

Finally, let's price the option with MLMC

In [42]:
def mlmc_l(l,N):
    S0 = 100
    sigma = 0.2
    r = 0.05
    T = 1
    K = 100
    dt = 2**(-l)
    sumY = 0
    sumY2 = 0
    for i in range(N):
        Sf = S0
        Sc = S0
        dWf = np.sqrt(dt) * np.random.randn(int(T/dt))
        for i in range(int(T/dt)):
            Sf += r * Sf * dt + sigma * Sf * dWf[i]
        Pf = np.max(Sf - K, 0)
        
        if l == 0 :
            Pc = 0
        else :
            #dWc = dWf.reshape(-1, 2)
            dWc = np.sqrt(dt) * np.random.randn(int(T/dt/2))
            for i in range(int(T/dt/2)):
                Sc += r * Sc * dt + sigma * Sc * dWc[i]
            Pc = np.max(Sc - K, 0)
        
        Y = Pf - Pc
        sumY += Y
        sumY2 += Y**2
        
    return sumY, sumY2

In [43]:
N0 = 1e04
eps = 1e-06
gamma = 1
alpha_0 = -1
beta_0 = -1

mlmc_price = mlmc(N0, eps, mlmc_l, alpha_0, beta_0, gamma)

C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1224474138.py:15: RuntimeWarning: invalid value encountered in true_divide
  ml = np.abs(suml[0,:]/Nl)
C:\Users\oussama\AppData\Local\Temp\ipykernel_17464\1224474138.py:16: RuntimeWarning: invalid value encountered in true_divide
  Vl = np.maximum(0, suml[1, :]/Nl - ml**2)


LinAlgError: SVD did not converge in Linear Least Squares